In [1]:
# conda install keras-tuner
import keras_tuner as kt
import pandas as pd
import numpy as np
import keras
from datetime import datetime
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras import losses
from keras.constraints import maxnorm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import seaborn as sns
sns.set()
from collections import Counter
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.metrics import mean_squared_error, make_scorer, accuracy_score, f1_score, classification_report
# import pickle
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
from keras.models import load_model
import matplotlib.pyplot as plt
from keras.callbacks import *


2022-12-03 11:06:49.423855: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### Bitcoin

In [3]:
regs = pd.read_csv("../Data/train_btc_selected_features.csv")
btc = pd.read_csv("../Data/btc_Data.csv")
for i in range(len(btc['Date'])):
    btc['Date'][i]  =  datetime.strptime(btc['Date'][i], '%Y/%m/%d')

btc = btc.set_index("Date")

/tmp/ipykernel_13680/2421463472.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc['Date'][i]  =  datetime.strptime(btc['Date'][i], '%Y/%m/%d')


In [4]:
btcData = btc[regs.columns]

In [5]:
btcData['returns'] = btcData['priceUSD'].pct_change()

/tmp/ipykernel_13680/751970969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btcData['returns'] = btcData['priceUSD'].pct_change()


In [6]:
Data = btcData.drop(columns=['priceUSD'])
Data = Data[1:]

In [7]:
Data 

,difficulty14mom,difficulty30mom,difficulty3rsi,difficulty90mom,difficulty90sma,hashrate90std,hashrate90var,median_transaction_fee90rocUSD,median_transaction_feeUSD,mining_profitability,...,price90wmaUSD,sentinusd90emaUSD,sentinusdUSD,top100cap,top100cap14trx,top100cap30rsi,top100cap30trx,top100cap90rsi,top100cap90trx,returns
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-18,1.621380e+02,1.641620e+02,100.00,1.737230e+02,2.055500e+01,8.384389e+08,1.757449e+17,0.000,0.000,401834.0000,...,0.0,0,2.612000e+03,25.782,0.139,71.191,0.308,79.756,0.550,0.466667
2010-07-19,1.621380e+02,1.641620e+02,100.00,1.737230e+02,2.248600e+01,8.819952e+08,1.944789e+17,0.000,0.000,481473.0000,...,0.0,0,4.047000e+03,25.685,0.123,68.863,0.300,78.999,0.547,0.183196
2010-07-20,1.584060e+02,1.641620e+02,100.00,1.735570e+02,2.441400e+01,9.280542e+08,2.153212e+17,0.000,0.000,431831.0000,...,0.0,0,2.341000e+03,25.602,0.107,66.923,0.292,78.355,0.544,-0.088475
2010-07-21,1.580420e+02,1.641620e+02,100.00,1.700790e+02,2.630400e+01,9.761949e+08,2.382391e+17,0.000,0.000,460783.0000,...,0.0,0,2.122000e+03,25.638,0.092,67.334,0.284,78.431,0.541,-0.020434
2010-07-22,1.580420e+02,1.641620e+02,100.00,1.700790e+02,2.819400e+01,1.019474e+09,2.598318e+17,0.000,0.000,334641.0000,...,0.0,0,2.570000e+03,25.604,0.078,66.526,0.275,78.166,0.537,-0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-24,1.882952e+11,1.148115e+11,99.86,8.598887e+12,3.384949e+13,4.612156e+19,5.317997e+38,165.380,0.818,0.0577,...,18820.0,28544682257,6.114577e+09,15.236,-0.080,40.118,-0.042,45.140,0.001,0.008075
2022-11-25,1.882952e+11,1.148115e+11,99.86,8.598887e+12,3.394503e+13,4.542600e+19,5.158804e+38,214.464,0.812,0.0541,...,18759.0,28076113671,7.224812e+09,15.231,-0.075,39.880,-0.043,45.047,0.001,-0.006565
2022-11-26,1.882952e+11,1.148115e+11,99.86,8.598887e+12,3.404058e+13,4.624135e+19,5.345655e+38,255.004,0.809,0.0571,...,18701.0,27566714445,4.898449e+09,15.233,-0.070,40.069,-0.043,45.106,0.001,0.004001


In [8]:
# divide X and Y
X = Data.iloc[:,0:]
#Y = Data['returns']   # 用returns的话就用这一行，然后把下一行comment掉
Y = btcData['priceUSD'].shift(-30)[1:] # 反之亦然

In [9]:
X.head(3)

,difficulty14mom,difficulty30mom,difficulty3rsi,difficulty90mom,difficulty90sma,hashrate90std,hashrate90var,median_transaction_fee90rocUSD,median_transaction_feeUSD,mining_profitability,...,price90wmaUSD,sentinusd90emaUSD,sentinusdUSD,top100cap,top100cap14trx,top100cap30rsi,top100cap30trx,top100cap90rsi,top100cap90trx,returns
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-18,162.138,164.162,100.0,173.723,20.555,838438881.0,1.757449e+17,0.0,0.0,401834.0,...,0.0,0,2612.0,25.782,0.139,71.191,0.308,79.756,0.550,0.466667
2010-07-19,162.138,164.162,100.0,173.723,22.486,881995244.0,1.944789e+17,0.0,0.0,481473.0,...,0.0,0,4047.0,25.685,0.123,68.863,0.300,78.999,0.547,0.183196
2010-07-20,158.406,164.162,100.0,173.557,24.414,928054231.0,2.153212e+17,0.0,0.0,431831.0,...,0.0,0,2341.0,25.602,0.107,66.923,0.292,78.355,0.544,-0.088475


In [10]:
# Split into three data sets
X_train = X['2016-01-01':'2019-12-31']
X_val = X['2020-01-01':'2021-05-31']
X_test = X['2021-06-01':'2023-01-01']

Y_train = Y['2016-01-01':'2019-12-31']
Y_val = Y['2020-01-01':'2021-05-31']
Y_test = Y['2021-06-01':'2023-01-01']

In [11]:
estimators=[]
estimators.append(['mixmax',MinMaxScaler()])
estimators.append(['robust',RobustScaler()])
scale=Pipeline(estimators,verbose=True)
scale.fit(X_train)

[Pipeline] ............ (step 1 of 2) Processing mixmax, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing robust, total=   0.0s


Pipeline(steps=[('mixmax', MinMaxScaler()), ['robust', RobustScaler()]],
         verbose=True)

In [12]:
X_train=scale.transform(X_train)
X_test=scale.transform(X_test)
X_val = scale.transform(X_val)

In [13]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [14]:
# define neural network model
shape=X.shape[1]
def build_model(hp, initializer='normal', activation='relu', NUM_FEATURES=shape):
    # create model
    hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
    model = Sequential()
    model.add(Dense(hp_units1, input_shape=(NUM_FEATURES,), kernel_initializer=initializer, activation=activation))
    model.add(Dense(hp_units2, activation=activation))
    model.add(Dense(hp_units3, activation=activation))
    model.add(Dense(1, activation=activation))
    # Compile model
    adam=keras.optimizers.Adam(lr=lr_schedule(0), amsgrad=True)
    #sgd=keras.optimizers.SGD(learning_rate=0.08, momentum=0.9, nesterov=False)
    model.compile(loss='logcosh', optimizer=adam, metrics=['mae'])
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',
    directory='btc_tune',
    project_name='ANN_TUNE'
)

tuner.search(X_train, Y_train, epochs=5000, validation_data=(X_val, Y_val))

2022-12-03 11:06:51.935927: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-03 11:06:51.936722: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
/home/spectre/anaconda3/envs/tensorplustorch/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Learning rate:  0.001

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
480               |?                 |units1
384               |?                 |units2
320               |?                 |units3

Learning rate:  0.001
Epoch 1/5000
46/46 [==============================] - 1s 15ms/step - loss: 4638.6973 - mae: 4639.3906 - val_loss: 18311.5977 - val_mae: 18312.2891
Epoch 2/5000
46/46 [==============================] - 0s 10ms/step - loss: 2424.6301 - mae: 2425.3230 - val_loss: 6763.4771 - val_mae: 6764.1709
Epoch 3/5000
46/46 [==============================] - 1s 17ms/step - loss: 1432.9191 - mae: 1433.6121 - val_loss: 6482.1313 - val_mae: 6482.8242
Epoch 4/5000
46/46 [==============================] - 0s 7ms/step - loss: 1198.4069 - mae: 1199.1002 - val_loss: 6855.1562 - val_mae: 6855.8496
Epoch 5/5000
46/46 [==============================] - 0s 7ms/step - loss: 1068.0933 - mae: 1068.7859 - val_loss: 7274.4834 - val_mae: 7275.1763
Epoch 6/5000
46

46/46 [==============================] - 0s 6ms/step - loss: 337.9890 - mae: 338.6802 - val_loss: 8157.9189 - val_mae: 8158.6123
Epoch 57/5000
46/46 [==============================] - 0s 5ms/step - loss: 352.1619 - mae: 352.8517 - val_loss: 8479.8291 - val_mae: 8480.5225
Epoch 58/5000
46/46 [==============================] - 1s 15ms/step - loss: 387.1444 - mae: 387.8362 - val_loss: 8302.6279 - val_mae: 8303.3203
Epoch 59/5000
46/46 [==============================] - 1s 19ms/step - loss: 317.6659 - mae: 318.3555 - val_loss: 7943.5581 - val_mae: 7944.2524
Epoch 60/5000
46/46 [==============================] - 1s 31ms/step - loss: 317.6264 - mae: 318.3179 - val_loss: 7777.7842 - val_mae: 7778.4771
Epoch 61/5000
46/46 [==============================] - 0s 9ms/step - loss: 315.1986 - mae: 315.8893 - val_loss: 7719.6948 - val_mae: 7720.3877
Epoch 62/5000
46/46 [==============================] - 1s 11ms/step - loss: 314.0262 - mae: 314.7172 - val_loss: 8104.6504 - val_mae: 8105.3433
Epoch 63/

46/46 [==============================] - 0s 6ms/step - loss: 264.9640 - mae: 265.6530 - val_loss: 7763.3340 - val_mae: 7764.0264
Epoch 114/5000
46/46 [==============================] - 1s 11ms/step - loss: 261.1137 - mae: 261.8031 - val_loss: 8087.6191 - val_mae: 8088.3125
Epoch 115/5000
46/46 [==============================] - 0s 10ms/step - loss: 288.4123 - mae: 289.1009 - val_loss: 7967.3672 - val_mae: 7968.0605
Epoch 116/5000
46/46 [==============================] - 1s 15ms/step - loss: 281.8394 - mae: 282.5279 - val_loss: 8966.3154 - val_mae: 8967.0088
Epoch 117/5000
46/46 [==============================] - 1s 24ms/step - loss: 302.9285 - mae: 303.6194 - val_loss: 8310.2188 - val_mae: 8310.9111
Epoch 118/5000
46/46 [==============================] - 0s 10ms/step - loss: 254.8239 - mae: 255.5132 - val_loss: 8052.4336 - val_mae: 8053.1270
Epoch 119/5000
46/46 [==============================] - 1s 14ms/step - loss: 251.9321 - mae: 252.6208 - val_loss: 8159.4331 - val_mae: 8160.1265
E

Epoch 170/5000
46/46 [==============================] - 0s 9ms/step - loss: 224.6377 - mae: 225.3265 - val_loss: 8115.2651 - val_mae: 8115.9575
Epoch 171/5000
46/46 [==============================] - 0s 7ms/step - loss: 243.3749 - mae: 244.0625 - val_loss: 8089.0762 - val_mae: 8089.7690
Epoch 172/5000
46/46 [==============================] - 0s 6ms/step - loss: 231.7369 - mae: 232.4268 - val_loss: 7702.0298 - val_mae: 7702.7236
Epoch 173/5000
46/46 [==============================] - 0s 8ms/step - loss: 231.4895 - mae: 232.1786 - val_loss: 7578.2485 - val_mae: 7578.9419
Epoch 174/5000
46/46 [==============================] - 0s 7ms/step - loss: 217.8037 - mae: 218.4916 - val_loss: 7627.5400 - val_mae: 7628.2324
Epoch 175/5000
46/46 [==============================] - 0s 11ms/step - loss: 246.2366 - mae: 246.9255 - val_loss: 8094.6821 - val_mae: 8095.3755
Epoch 176/5000
46/46 [==============================] - 0s 7ms/step - loss: 231.2961 - mae: 231.9837 - val_loss: 8364.7500 - val_mae: 8

...

46/46 [==============================] - 1s 12ms/step - loss: 95.1750 - mae: 95.8614 - val_loss: 8175.0850 - val_mae: 8175.7783
Epoch 1079/5000
46/46 [==============================] - 0s 8ms/step - loss: 98.3087 - mae: 98.9948 - val_loss: 8106.1929 - val_mae: 8106.8857
Epoch 1080/5000
46/46 [==============================] - 0s 7ms/step - loss: 100.7211 - mae: 101.4074 - val_loss: 8082.7871 - val_mae: 8083.4795
Epoch 1081/5000
46/46 [==============================] - 1s 11ms/step - loss: 107.6753 - mae: 108.3614 - val_loss: 8030.7168 - val_mae: 8031.4097
Epoch 1082/5000
46/46 [==============================] - 1s 12ms/step - loss: 118.4845 - mae: 119.1704 - val_loss: 7964.7778 - val_mae: 7965.4712
Epoch 1083/5000
46/46 [==============================] - 1s 14ms/step - loss: 102.2693 - mae: 102.9568 - val_loss: 7901.6782 - val_mae: 7902.3721
Epoch 1084/5000
46/46 [==============================] - 0s 6ms/step - loss: 94.0268 - mae: 94.7113 - val_loss: 8083.5972 - val_mae: 8084.2891
Epo

46/46 [==============================] - 1s 14ms/step - loss: 95.5629 - mae: 96.2451 - val_loss: 7961.5674 - val_mae: 7962.2612
Epoch 1135/5000
46/46 [==============================] - 1s 18ms/step - loss: 109.3632 - mae: 110.0480 - val_loss: 8299.4941 - val_mae: 8300.1875
Epoch 1136/5000
46/46 [==============================] - 0s 9ms/step - loss: 114.5876 - mae: 115.2722 - val_loss: 8125.8413 - val_mae: 8126.5347
Epoch 1137/5000
46/46 [==============================] - 0s 9ms/step - loss: 115.7072 - mae: 116.3914 - val_loss: 8013.5898 - val_mae: 8014.2822
Epoch 1138/5000
46/46 [==============================] - 0s 9ms/step - loss: 106.1780 - mae: 106.8591 - val_loss: 8006.2354 - val_mae: 8006.9287
Epoch 1139/5000
46/46 [==============================] - 0s 8ms/step - loss: 95.2672 - mae: 95.9501 - val_loss: 8091.8789 - val_mae: 8092.5728
Epoch 1140/5000
46/46 [==============================] - 1s 12ms/step - loss: 101.7281 - mae: 102.4116 - val_loss: 7928.6250 - val_mae: 7929.3179
Ep

46/46 [==============================] - 0s 10ms/step - loss: 102.2739 - mae: 102.9582 - val_loss: 7946.2739 - val_mae: 7946.9668
Epoch 1191/5000
46/46 [==============================] - 1s 16ms/step - loss: 112.9587 - mae: 113.6482 - val_loss: 8268.8906 - val_mae: 8269.5830
Epoch 1192/5000
46/46 [==============================] - 1s 16ms/step - loss: 108.8794 - mae: 109.5658 - val_loss: 8073.3511 - val_mae: 8074.0444
Epoch 1193/5000
46/46 [==============================] - 1s 14ms/step - loss: 107.1282 - mae: 107.8145 - val_loss: 8112.0889 - val_mae: 8112.7822
Epoch 1194/5000
46/46 [==============================] - 1s 21ms/step - loss: 108.2036 - mae: 108.8900 - val_loss: 8057.8657 - val_mae: 8058.5586
Epoch 1195/5000
46/46 [==============================] - 1s 13ms/step - loss: 106.0878 - mae: 106.7740 - val_loss: 8062.0981 - val_mae: 8062.7910
Epoch 1196/5000
46/46 [==============================] - 1s 11ms/step - loss: 107.1661 - mae: 107.8517 - val_loss: 7972.4106 - val_mae: 7973

Epoch 1247/5000
46/46 [==============================] - 0s 9ms/step - loss: 109.4211 - mae: 110.1067 - val_loss: 8139.4351 - val_mae: 8140.1274
Epoch 1248/5000
46/46 [==============================] - 0s 6ms/step - loss: 104.9647 - mae: 105.6509 - val_loss: 8182.1963 - val_mae: 8182.8896
Epoch 1249/5000
46/46 [==============================] - 0s 10ms/step - loss: 112.4359 - mae: 113.1206 - val_loss: 8153.8726 - val_mae: 8154.5659
Epoch 1250/5000
46/46 [==============================] - 0s 9ms/step - loss: 103.6702 - mae: 104.3527 - val_loss: 8139.8540 - val_mae: 8140.5474
Epoch 1251/5000
46/46 [==============================] - 0s 6ms/step - loss: 92.9348 - mae: 93.6201 - val_loss: 8137.0176 - val_mae: 8137.7109
Epoch 1252/5000
46/46 [==============================] - 0s 6ms/step - loss: 108.9734 - mae: 109.6584 - val_loss: 8076.5957 - val_mae: 8077.2891
Epoch 1253/5000
46/46 [==============================] - 1s 15ms/step - loss: 96.8806 - mae: 97.5645 - val_loss: 8085.4141 - val_ma

Epoch 1304/5000
46/46 [==============================] - 0s 9ms/step - loss: 91.3090 - mae: 91.9948 - val_loss: 8237.7637 - val_mae: 8238.4570
Epoch 1305/5000
46/46 [==============================] - 1s 11ms/step - loss: 93.2205 - mae: 93.9037 - val_loss: 8113.3896 - val_mae: 8114.0830
Epoch 1306/5000
46/46 [==============================] - 0s 9ms/step - loss: 97.7292 - mae: 98.4156 - val_loss: 8024.0425 - val_mae: 8024.7358
Epoch 1307/5000
46/46 [==============================] - 1s 17ms/step - loss: 99.1963 - mae: 99.8815 - val_loss: 7998.0024 - val_mae: 7998.6953
Epoch 1308/5000
46/46 [==============================] - 0s 8ms/step - loss: 110.4471 - mae: 111.1251 - val_loss: 8302.7031 - val_mae: 8303.3945
Epoch 1309/5000
46/46 [==============================] - 0s 11ms/step - loss: 121.6295 - mae: 122.3142 - val_loss: 8064.1973 - val_mae: 8064.8901
Epoch 1310/5000
46/46 [==============================] - 1s 14ms/step - loss: 106.4161 - mae: 107.0994 - val_loss: 8175.8809 - val_mae:

In [ ]:
# Get the best hyperparameters.
best_hp = tuner.get_best_hyperparameters()[0]
# Build the model with the best hp.
regressor = build_model(best_hp)
# Fit with the entire dataset.
X_all = np.concatenate((X_train, X_val))
Y_all = np.concatenate((Y_train, Y_val))


In [ ]:
# best hyperparameters
best_hp

stop training when val_loss is not increasing

In [ ]:
earlyStopping = EarlyStopping(monitor='loss', patience=1000,verbose=1, mode='auto')

In [ ]:
regressor.fit(x=X_all, y=Y_all, epochs=5000, use_multiprocessing=True, callbacks=[earlyStopping])

In [ ]:

y_pred=regressor.predict(X_test)

In [ ]:
# for check
Y_train_pred=regressor.predict(X_train)
r2_score(Y_train, Y_train_pred) #training score

In [ ]:
r2=r2_score(Y_test[:-30],y_pred[:-30]) #score/ r^2
print(f'r2:{r2}')

In [ ]:
# r2_oos
def r2_oos(ret, pred):
    sum_of_sq_res = np.nansum(np.power((ret-pred), 2))
    sum_of_sq_total = np.nansum(np.power(ret, 2))
    
    return 1-sum_of_sq_res/sum_of_sq_total

In [ ]:
r2_oos = r2_oos(Y_test[:-30], y_pred[:-30])
print(f'r2_oos:{r2_oos}')

In [ ]:
mae=mean_absolute_error(Y_test[:-30],y_pred[:-30]) #mae
print(f'mae:{mae}')

rmse=np.sqrt(mean_squared_error(Y_test[:-30],y_pred[:-30])) #rmse
print(f'rmse:{rmse}')

mape=mean_absolute_percentage_error(Y_test[:-30],y_pred[:-30]) #mape
print(f'mape:{mape}')

-----------------------------

In [ ]:
pre_df = pd.DataFrame(zip(Y_test,y_pred),columns=['Y_test','y_pred'])
pre_df.index = Y_test.index
pre_df

In [ ]:
pre_df['pred_returns'] = pre_df['y_pred'].pct_change()

In [ ]:
pre_df

In [ ]:
pd.DataFrame(zip(Y_test,y_pred),columns=['Y_test','y_pred']).plot()

In [ ]:
pre_df.to_csv("../result/ANN/btc_NN.csv")

In [ ]:
!kdeconnect-cli -n TAS-AN00 --ping-msg 'Script complete!'